In [1]:
from gpflowopt.acquisition import Acquisition
from gpflowopt.pareto import Pareto

from gpflow.param import DataHolder
from gpflow import settings

import numpy as np
import tensorflow as tf


stability = settings.numerics.jitter_level
float_type = settings.dtypes.float_type

/Users/quannguyen/anaconda3/envs/BayesFAI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/quannguyen/anaconda3/envs/BayesFAI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/quannguyen/anaconda3/envs/BayesFAI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/quannguyen/anacond

In [ ]:
class HVPoIEx(Acquisition):
    def __init__(self, models):
        super(HVPoIEx, self).__init__(models)
        num_objectives = self.data[1].shape[1]
        
        self.pareto = Pareto(np.empty((0, num_objectives)))
        self.reference = DataHolder(np.ones((1, num_objectives)))
    
    def _estimate_reference(self):
        pf = self.pareto.front.value
        f_max = np.max(pf, axis=0, keepdims=True)
        f = f_max - np.min(pf, axis=0, keepdims=True)
        
        return f_max + 2 * f / pf.shape[0]
    
    def _setup(self):
        super(HVPoIEx, self)._setup()
        
        feasible_samples = self.data[0][self.highest_parent.feasible_data_index(), :]
        F = np.hstack((m.predict_f(feasible_samples)[0] for m in self.models))
        self.pareto.update(F)
        
        self.reference = self._estimate_reference()
        
    def build_acquisition(self, Xcand):
        